<a href="https://colab.research.google.com/github/vbsteja/facial_emotion_detection/blob/master/kaggle_facial_expression_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PreProcessing

In [0]:
#!wget https://cdn.pixabay.com/photo/2014/11/30/14/11/kitty-551554__340.jpg
!pip install tensorflow-gpu==1.15
!pip install --user kaggle
!rm -r /root/.kaggle
!mkdir /root/.kaggle
!echo '{"username":"vbsuryateja2","key":"***************"}' > /root/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
#!unzip test1.zip
#!unzip train.zip
!mkdir weights

     |████████████████████████████████| 411.5MB 40kB/s 
     |████████████████████████████████| 512kB 55.4MB/s 
     |████████████████████████████████| 3.8MB 51.7MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-gpu 1.14.0
    Uninstalling tensorflow-gpu-1.14.0:
      Successfully uninstalled tensorflow-gpu-1.14.0


example_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
fer2013.tar.gz: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘weights’: File exists


In [0]:
!tar -xvf  fer2013.tar.gz

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/


In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import ELU,Softmax
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.optimizers import Adam
#from config import emotion_config as config
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import argparse
import os
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
from keras.utils import to_categorical


### Model Definition

In [0]:
class EmotionVGGNet:

	@staticmethod
	def build(width, height, depth, classes):
		model = Sequential()
		inputShape = (height,width,depth)
		chanDim = -1

		if K.image_data_format == "channels_first":
			inputShape = (depth,height,width)
			chanDim = 1
		model = Sequential([
			Conv2D(32,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(32,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Conv2D(64,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(64,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Conv2D(64,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(64,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Flatten(),
			Dense(64,kernel_initializer="he_normal"),
			ELU(),
			BatchNormalization(),
			Dropout(0.25),
			Dense(64,kernel_initializer="he_normal"),
			ELU(),
			BatchNormalization(),
			Dropout(0.25),
			Dense(classes,kernel_initializer="he_normal"),
			Softmax()])
		return model


### Data preparation


In [19]:
#CONST
emotion_dict = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
BASE_DIR = "fer2013/"
data = pd.read_csv(BASE_DIR+"fer2013.csv")
data.loc[data["emotion"]==1,["emotion"]] = 0
data.head()


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
train_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "Training"]["pixels"].values]
train_pixels = np.array([i.reshape((48,48)) for i in train_pixels])
train_pixels = np.expand_dims(train_pixels,-1)
train_labels = np.array([int(j) for j in data[data.Usage == 'Training']["emotion"].values])
train_labels = np.array(to_categorical(train_labels,num_classes = 7))

In [18]:
data.emotion.unique()

array([0, 2, 4, 6, 3, 5])

In [0]:
test_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "PrivateTest"]["pixels"].values]
test_pixels = np.array([i.reshape((48,48)) for i in test_pixels])
test_pixels = np.expand_dims(test_pixels,-1)
test_labels = np.array([int(j) for j in data[data.Usage == 'PrivateTest']["emotion"].values])
test_labels = np.array(to_categorical(test_labels,num_classes = 7))

In [0]:
validation_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "PublicTest"]["pixels"].values]
validation_pixels = np.array([i.reshape((48,48)) for i in validation_pixels])
validation_pixels = np.expand_dims(validation_pixels,-1)
validation_labels = np.array([int(j) for j in data[data.Usage == 'PublicTest']["emotion"].values])
validation_labels = np.array(to_categorical(validation_labels,num_classes = 7))

In [0]:
train = ImageDataGenerator(rotation_range=10, zoom_range=0.1,horizontal_flip=True, rescale=1 / 255.0, fill_mode="nearest")
test = ImageDataGenerator(rescale=1 / 255.0)
validation = ImageDataGenerator(rescale=1 / 255.0)

### Model Training

In [0]:
print("[INFO] compiling model...")
model = EmotionVGGNet.build(width=48, height=48, depth=1,classes=len(emotion_dict))

In [30]:
opt = Adam(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

checkpointer = ModelCheckpoint(filepath='weights/EmotionVGGNet.hdf5', verbose=1, save_best_only=True)

model.fit_generator(
	train.flow(train_pixels,train_labels,batch_size=32),
	validation_data = validation.flow(validation_pixels,validation_labels),
	epochs = 30,
	callbacks = [checkpointer],
	verbose=1)


Epoch 1/30
898/898 [==============================] - 20s 22ms/step - loss: 1.0498 - acc: 0.6044 - val_loss: 1.0465 - val_acc: 0.6135

Epoch 00001: val_loss improved from inf to 1.04645, saving model to weights/EmotionVGGNet.hdf5
Epoch 2/30
898/898 [==============================] - 17s 19ms/step - loss: 1.0185 - acc: 0.6187 - val_loss: 1.0574 - val_acc: 0.6133

Epoch 00002: val_loss did not improve from 1.04645
Epoch 3/30
898/898 [==============================] - 17s 18ms/step - loss: 1.0146 - acc: 0.6214 - val_loss: 1.0313 - val_acc: 0.6213

Epoch 00003: val_loss improved from 1.04645 to 1.03134, saving model to weights/EmotionVGGNet.hdf5
Epoch 4/30
898/898 [==============================] - 17s 19ms/step - loss: 1.0042 - acc: 0.6257 - val_loss: 1.0386 - val_acc: 0.6197

Epoch 00004: val_loss did not improve from 1.03134
Epoch 5/30
898/898 [==============================] - 16s 18ms/step - loss: 0.9970 - acc: 0.6262 - val_loss: 1.0146 - val_acc: 0.6236

Epoch 00005: val_loss improve

### Model Evaluation

In [28]:
#model = load_model("weights/EmotionVGGNet.hdf5")

loss, acc = model.evaluate_generator(
test.flow(test_pixels,test_labels))

print("[INFO] accuracy : {:.2f}".format(acc*100))


[INFO] accuracy : 67.85
